In [1]:
#Using the spark session to get an sql 
import pyspark as ps
spark = ps.sql.SparkSession.builder.getOrCreate()

In [5]:
#Import other useful libraries

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#This provides us with a way of getting an sql dataframe with a schema that structures the data
bigdata = spark.read.csv(r'C:\Users\matom\OneDrive\Documents\Data Science\MIT Big Data\MIT805\multiproductdata.csv', header = True, inferSchema=True)

**Exploratory Data Analysis**

In [3]:
#Now we can see the schema of the dataframe using the following command
bigdata.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [6]:
#Let's check the description of our dataset
bigdata.describe().toPandas()

,summary,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,count,67501979,67501979,67501979,45603808,58283744,67501979,67501979,67501969
1,mean,None,1.2514064889882294E7,2.0578976443190984E18,None,NaN,292.45931656461437,5.386397456296759E8,None
2,stddev,None,1.725741362984622E7,2.0125490328842856E16,None,NaN,355.67449958606784,2.288516105152206E7,None
3,min,cart,1000365,2053013552226107603,accessories.bag,a-case,0.0,10300217,0000007c-adbf-4ed7-af17-d1fef9763d67
4,max,view,100028554,2187707861038006932,stationery.cartrige,zyxel,2574.07,579969851,fffffde2-4522-4b44-8a32-510c55739ba1


In [7]:
#Let's remove null values from our dataset, since they will not provide us with any sort of insight
clean_bigdata = bigdata.dropna(how='any')

In [4]:
#Length of the columns in my dataframe
len(bigdata.columns)

9

In [5]:
#Check the count of the rows in my original dataframe
bigdata.count()

67501979

In [ ]:
#Check the count of the rows in my dataframe without any nulls
clean_bigdata.count()

In [6]:
bigdata.select("event_time","brand","user_session").show(5)

+-------------------+------+--------------------+
|         event_time| brand|        user_session|
+-------------------+------+--------------------+
|2019-11-01 02:00:00|xiaomi|4d3b30da-a5e4-49d...|
|2019-11-01 02:00:00|janome|8e5f4f83-366c-4f7...|
|2019-11-01 02:00:01| creed|755422e7-9040-477...|
|2019-11-01 02:00:01|    lg|3bfb58cd-7892-48c...|
|2019-11-01 02:00:01|xiaomi|313628f1-68b8-460...|
+-------------------+------+--------------------+
only showing top 5 rows



In [7]:
#Now we want to see how many unique user sessions do we have in our dataset
Number_unique_user_sessions = bigdata.select("user_session").distinct().count()
print(f"Total number of unique user sessions: {Number_unique_user_sessions}")

Total number of unique user sessions: 13776051


In [8]:
#Now we want to understand for the time period under scrutiny what is the total number of unique customers that visited our site
Number_unique_user_id = bigdata.select("user_id").distinct().count()
print(f"Total number of unique user sessions: {Number_unique_user_id}")

Total number of unique user sessions: 3696117


In [9]:
#Now we want to check the price range and stastical measures for products on our site
bigdata.describe("price").show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|          67501979|
|   mean|292.45931656461437|
| stddev|355.67449958606784|
|    min|               0.0|
|    max|           2574.07|
+-------+------------------+



In [10]:
#We can also look at the brands and events associated with each brand
bigdata.groupby("brand").count().show()

+-------------+------+
|        brand| count|
+-------------+------+
|     yokohama|128697|
|      edifier|  3565|
|        welss|  4929|
|      tuffoni|  3617|
|         tega|  2317|
|      serebro|  7105|
|        crest|   325|
|         tmnt|   574|
|       alutec|  2587|
|        sonel| 11501|
|     nocnezna|   250|
|     nutricia|  2496|
|      bombbar|   852|
|trianglegroup|    97|
|      keenway|  2368|
|        sigma|  2424|
|        goo.n|  1031|
|      fitwell|   532|
| belaakalitva|   218|
|       marley|  1063|
+-------------+------+
only showing top 20 rows



In [ ]:
#We can also look at the spend by category_code and by brand, but first we need to see what event actually occurred. Only Event = purchase will be considered.
bigdata.groupby("event_type","price").sum().show()